В задаче нужно построить классификатор, который может определить, является ли слово фамилией или нет. Обучающая выборка - список слов, размеченный 0 и 1 (1 - если является фамилией, 0 иначе).

In [1]:
import pandas as pd 
import numpy as np
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import roc_auc_score
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.model_selection import train_test_split

In [2]:
train = pd.read_csv('train.txt', header=0, sep=',', encoding = 'utf-8')
train.dtypes

Word          object
is_surname     int64
dtype: object

In [3]:
train_data, test_data, train_labels, test_labels = train_test_split(train.Word, train.is_surname, test_size = 0.1, stratify = train.is_surname)

In [4]:
train_data.head(5)

22687      Долларом
62187       ПОДОБИИ
88959         Уайлс
19553       дверцах
79210    слабоволия
Name: Word, dtype: object

Составляем список всех слов

In [5]:
corpus = []
L = len(train_data)
for i in range(L):
    corpus.append(train_data.iloc[i])

In [6]:
print(corpus[:10])

['Долларом', 'ПОДОБИИ', 'Уайлс', 'дверцах', 'слабоволия', 'ДЕЛОПРОИЗВОДИТЕЛЬ', 'мореходам', 'молитвеннику', 'Ингмару', 'Александрам']


In [7]:
from sklearn.pipeline import Pipeline
from sklearn.svm import SVC, LinearSVC

Основная идея решения - использовать в качестве признаков слов их n-граммы (по сути one-hot-encoding). Для разбиения используем CountVectorizer. Размер n-грамм выберем от 1 до 8.

In [10]:
pipeline = Pipeline([("vectorizer", CountVectorizer(analyzer = 'char_wb', min_df=1, ngram_range=(1, 8), lowercase = False)), ("algo", LogisticRegression(penalty = 'l2', max_iter = 300, solver = 'lbfgs'))])

In [11]:
pipeline.fit(train_data, train_labels)

Pipeline(steps=[('vectorizer', CountVectorizer(analyzer='char_wb', binary=False, decode_error='strict',
        dtype=<class 'numpy.int64'>, encoding='utf-8', input='content',
        lowercase=False, max_df=1.0, max_features=None, min_df=1,
        ngram_range=(1, 8), preprocessor=None, stop_words=None,
   ...enalty='l2', random_state=None, solver='lbfgs', tol=0.0001,
          verbose=0, warm_start=False))])

In [12]:
print(roc_auc_score(test_labels, pipeline.predict_proba(test_data)[:,1]))

0.92310617764


Также заметим, что дополнительно можно использовать, например, такие признаки: является ли первая буква заглавной, написано ли все слово капсом. Можель была протестирована с этими признаками. Качество на кросс валидации почти не улучшилось.

На кросс-валидации хорошее качество. Можно запускать алгоритм на тестовых данных.

Аналогично из тестовых данных делаем корпус слов и передаем в уже обученный pipeline.

In [13]:
real_df = pd.read_csv("test.txt")

In [14]:
real_data = real_df.Word

In [15]:
real_data.head(5)

0    Аалто
1      ААР
2     Аара
3     Ааре
4    Аарон
Name: Word, dtype: object

In [16]:
real_corpus = []
L = len(real_data)
for i in range(L):
    real_corpus.append(real_data.iloc[i].lower())

Метрика в задаче - плошадь под AUC-ROC кривой. Поэтому нам нужны вероятности принадлежности слов из теста к классу 1 (фамилии).

In [17]:
preds = pipeline.predict_proba(real_corpus)

In [18]:
print(preds[:10])

[[ 0.92661493  0.07338507]
 [ 0.9540203   0.0459797 ]
 [ 0.95914389  0.04085611]
 [ 0.98913298  0.01086702]
 [ 0.93468895  0.06531105]
 [ 0.96019346  0.03980654]
 [ 0.95782701  0.04217299]
 [ 0.95580892  0.04419108]
 [ 0.95580892  0.04419108]
 [ 0.96380841  0.03619159]]


In [19]:
print(len(real_df))

188920


In [20]:
print(len(preds))

188920


In [22]:
real_df['Id'] = np.arange(len(real_df))
real_df['Answer'] = preds[:, 1] 
real_df.head(21)

,Word,Id,Answer
0,Аалто,0,0.073385
1,ААР,1,0.045980
2,Аара,2,0.040856
3,Ааре,3,0.010867
4,Аарон,4,0.065311
5,Аароне,5,0.039807
6,Ааронов,6,0.042173
7,Аароном,7,0.044191
8,Аароном,8,0.044191
9,Аарону,9,0.036192


Записываем в файл, оставив только id слова.

In [23]:
real_df = real_df.drop('Word', 1)
real_df.to_csv("subm.txt", sep=',', index=False)